<a href="https://colab.research.google.com/github/tapiwamesa/Amini-Soil-Prediction/blob/main/Amini_Soil_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# importing dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
from sklearn.preprocessing import StandardScaler

In [10]:
# mounting drive

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# importing the data

train_gap = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data /Zindi | Soil Nutrients Prediction/Gap_Train.csv")
test_gap = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data /Zindi | Soil Nutrients Prediction/Gap_Test.csv")
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data /Zindi | Soil Nutrients Prediction/Train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data /Zindi | Soil Nutrients Prediction/Test.csv")

In [28]:
# viewing the data

train_gap.head()

,Nutrient,Required,Available,Gap,PID
0,N,100.0,3796.0000,-3696.0000,ID_I5RGjv
1,P,40.0,0.9928,39.0072,ID_I5RGjv
2,K,52.0,429.2400,-377.2400,ID_I5RGjv
3,Ca,12.0,19943.6000,-19931.6000,ID_I5RGjv
4,Mg,8.0,6745.2000,-6737.2000,ID_I5RGjv


In [14]:
train.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,...,0.34,147,6830,2310,5.66,75.2,85.0,0.82,2.98,0.24
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,...,11.70,151,1180,235,19.40,96.2,409.0,2.57,4.32,0.10
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,...,21.80,151,1890,344,11.00,76.7,65.0,1.95,1.24,0.22
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,...,39.90,201,6660,719,14.90,81.9,73.0,4.90,3.08,0.87
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,...,1.00,90,7340,1160,8.66,69.4,149.0,0.55,3.03,0.31


In [13]:
test_gap.head()

,Nutrient,Required,PID
0,N,100.0,ID_NGS9Bx
1,P,40.0,ID_NGS9Bx
2,K,52.0,ID_NGS9Bx
3,Ca,12.0,ID_NGS9Bx
4,Mg,8.0,ID_NGS9Bx


In [15]:
test.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity
0,site_id_hgJpkz,ID_NGS9Bx,69.170794,44.522885,6.86,144,256,910,108,186,...,37.940418,467.619293,6.825,1.056416,25.50,15.25,8.732471,0.016981,0.005831,1.20
1,site_id_olmuI5,ID_YdVKXw,68.885265,44.741057,7.08,129,260,851,110,187,...,35.961353,542.590149,6.725,0.730379,18.75,14.00,10.565657,0.021030,0.005134,1.24
2,site_id_PTZdJz,ID_MZAlfE,68.970210,44.675777,6.50,142,259,901,109,187,...,38.983898,416.385437,6.825,1.146542,21.00,14.00,9.590125,0.018507,0.004480,1.23
3,site_id_DOTgr8,ID_GwCCMN,69.068751,44.647707,6.82,142,261,847,109,187,...,39.948471,374.971008,6.725,0.567210,23.25,12.25,9.669279,0.021688,0.006803,1.22
4,site_id_1rQNvy,ID_K8sowf,68.990002,44.577607,6.52,145,253,1109,110,186,...,33.658615,361.233643,6.200,1.169207,26.25,18.25,7.895920,0.023016,0.000874,1.23


In [21]:
# checking columns in test dataset
test.columns

Index(['site', 'PID', 'lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15',
       'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn',
       'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope',
       'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity'],
      dtype='object')

In [22]:
# checking columns in train dataset
train.columns

Index(['site', 'PID', 'lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15',
       'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn',
       'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope',
       'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity', 'N', 'P', 'K',
       'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B'],
      dtype='object')

## **2. Exploratory Data Analysis**

In [29]:
# data types and missing values for test dataset
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2418 entries, 0 to 2417
Data columns (total 33 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site         2418 non-null   object 
 1   PID          2418 non-null   object 
 2   lon          2418 non-null   float64
 3   lat          2418 non-null   float64
 4   pH           2418 non-null   float64
 5   alb          2418 non-null   int64  
 6   bio1         2418 non-null   int64  
 7   bio12        2418 non-null   int64  
 8   bio15        2418 non-null   int64  
 9   bio7         2418 non-null   int64  
 10  bp           2418 non-null   float64
 11  cec20        2418 non-null   float64
 12  dows         2418 non-null   float64
 13  ecec20       2418 non-null   float64
 14  hp20         2418 non-null   float64
 15  ls           2418 non-null   float64
 16  lstd         2418 non-null   float64
 17  lstn         2418 non-null   float64
 18  mb1          2418 non-null   float64
 19  mb2   

In [43]:
# Null values in test set
test.isnull().sum()[test.isnull().sum() != 0]

,0


In [30]:
# data types and missing values for train

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7744 entries, 0 to 7743
Data columns (total 44 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site         7744 non-null   object 
 1   PID          7744 non-null   object 
 2   lon          7744 non-null   float64
 3   lat          7744 non-null   float64
 4   pH           7744 non-null   float64
 5   alb          7744 non-null   int64  
 6   bio1         7744 non-null   int64  
 7   bio12        7744 non-null   int64  
 8   bio15        7744 non-null   int64  
 9   bio7         7744 non-null   int64  
 10  bp           7744 non-null   float64
 11  cec20        7744 non-null   float64
 12  dows         7744 non-null   float64
 13  ecec20       7739 non-null   float64
 14  hp20         7739 non-null   float64
 15  ls           7744 non-null   float64
 16  lstd         7744 non-null   float64
 17  lstn         7744 non-null   float64
 18  mb1          7744 non-null   float64
 19  mb2   

In [42]:
# Null values in train set
train.isnull().sum()[train.isnull().sum() != 0]

,0
ecec20,5
hp20,5
xhp20,5
BulkDensity,4


In [50]:
# filling the null values in train set

for column in train.columns:
    if train[column].isnull().any():
        train[column].fillna(np.mean(train[column]), inplace = True)

<ipython-input-50-b48c753299bc>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[column].fillna(np.mean(train[column]), inplace = True)


In [52]:
# checking if all nulls have been filled with column averages
train.isnull().sum()[train.isnull().sum() != 0]

,0


### **i. Lets explore the train set**

In [53]:
# Lets explore the train set

train.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,...,0.34,147,6830,2310,5.66,75.2,85.0,0.82,2.98,0.24
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,...,11.70,151,1180,235,19.40,96.2,409.0,2.57,4.32,0.10
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,...,21.80,151,1890,344,11.00,76.7,65.0,1.95,1.24,0.22
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,...,39.90,201,6660,719,14.90,81.9,73.0,4.90,3.08,0.87
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,...,1.00,90,7340,1160,8.66,69.4,149.0,0.55,3.03,0.31


In [54]:
# dropping the site and PID columns
train_set = train.drop(columns = ["site", "PID"])

In [59]:
# target columns

target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
train_data = train_set.drop(columns = target_columns)

In [61]:
# viewing clean train data

train_data.head()

,lon,lat,pH,alb,bio1,bio12,bio15,bio7,bp,cec20,...,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity
0,70.603761,46.173798,7.75,176,248,920,108,190,0.581573,22.00,...,20.544283,126.835480,7.050,1.962921,39.00,9.75,7.962668,0.016853,0.000708,1.46
1,70.590479,46.078924,7.10,181,250,1080,113,191,0.707011,24.00,...,18.869566,109.835541,6.975,0.162065,40.00,8.00,8.439500,0.018321,0.001676,1.52
2,70.582553,46.048820,6.95,188,250,1109,111,191,0.362439,15.25,...,24.719807,214.385269,6.725,0.744845,46.00,9.25,8.289246,0.020588,0.003885,1.46
3,70.573267,46.021910,7.83,174,250,1149,112,191,0.531739,22.00,...,27.230274,255.713043,6.625,0.708708,43.75,10.00,8.666523,0.016913,0.001714,1.48
4,70.585330,46.204336,8.07,188,250,869,114,191,0.039202,14.75,...,20.434782,86.220909,6.700,0.634153,49.25,7.00,15.139549,0.019791,0.000000,1.43


In [62]:
train_data.shape

(7744, 31)

In [63]:
test.shape

(2418, 33)

## **3. Model Building**

In [87]:
# importing metrics

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [83]:
# splitting train data into feature and target variables

X = train_data
y = train_set[target_columns]
X_test = test.drop(columns  = ["site", "PID"])

In [78]:
# Splitting the data into train and test

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

### **a. Linear Regression**

In [94]:
# importing linear regression
from sklearn.linear_model import LinearRegression

In [96]:
# Scaling the data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [98]:
# Training the linear regression on scaled data

linear = MultiOutputRegressor(LinearRegression())
linear.fit(X_train_scaled, y_train)

MultiOutputRegressor(estimator=LinearRegression())

In [106]:
# predicting the validation set

y_pred_linear = linear.predict(X_val_scaled)
mse_linear = mean_absolute_error(y_val, y_pred_linear)
rmse_linear = np.sqrt(mse_linear)
print("MSE:", mse_linear)
print("\nRMSE:", rmse_linear)

MSE: 214.04092670467008

RMSE: 14.630137617420763


### **b. Random Forest**

In [79]:
# Training the model

forest = MultiOutputRegressor(RandomForestRegressor(n_estimators = 100, random_state = 42))
forest.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [104]:
# predicting the validation set

y_pred_forest = forest.predict(X_val)
mse_forest = mean_absolute_error(y_val, y_pred_forest)
rmse_forest = np.sqrt(mse_forest)
print("MAE:", mse_forest)
print("\n RMSE", rmse_forest)

MAE: 159.88389656669992

 RMSE 12.644520416635022


### **c. LightGBM**

In [102]:
# Training the model

lgbm = MultiOutputRegressor(lgbm.LGBMRegressor(random_state = 42))
lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6693
[LightGBM] [Info] Number of data points in the train set: 6195, number of used features: 31
[LightGBM] [Info] Start training from score 1659.143341
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003917 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6693
[LightGBM] [Info] Number of data points in the train set: 6195, number of used features: 31
[LightGBM] [Info] Start training from score 15.498505
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003989 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6693
[LightGBM] [Info] Number of data points in the train set: 6195, number of used features: 31
[LightGBM] [Info] Start tr

MultiOutputRegressor(estimator=LGBMRegressor(random_state=42))

In [107]:
# predicting the validation set

y_pred_lgbm = lgbm.predict(X_val)
mse_lgbm = mean_absolute_error(y_val, y_pred_lgbm)
rmse_lgbm = np.sqrt(mse_lgbm)
print("MAE:", mse_lgbm)
print("\n RMSE", rmse_lgbm)

MAE: 158.05734207181388

 RMSE 12.57208582820742
